# Algorithms & Data Structures 2
Jurjen Verbruggen

# Graph generation

Below you can find several sliders to generate a nice graph. $n$ = the amount of nodes in the graph, and $p$ is the probability of generating an edge between any two nodes. Click on the Update button to generate a new graph given the two parameters, and click on Connect to join a disconnected graph together.

In [38]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from vertex_manipulations import *
import random
import networkx as nx
import matplotlib.pyplot as plt
from vcover import *

n_input = widgets.IntSlider(value = 15, min=1, max=100, step=1, description="n")
p_input = widgets.FloatSlider(value = 0.5, min=0, max=1, step=0.01, description="p")
update_button = widgets.Button(description="Reroll")
connect_button = widgets.Button(description="Connect", disabled=True)

pendant_plus_button = widgets.Button(description="p++")
pendant_minus_button = widgets.Button(description="p--")
tops_plus_button = widgets.Button(description="t++")
tops_minus_button = widgets.Button(description="t--")
highlight_kernelization_button = widgets.Button(description="Refresh")

k_vcover_input_max = n_input.value
k_vcover_input = widgets.IntSlider(value=1, min=1, max=k_vcover_input_max, step=1, description="k")
calculate_k_vcover_button = widgets.Button(description="Calculate", disabled=True)
calculated_text = widgets.Textarea(description="calculated")
calculated_one_text = widgets.Textarea()
progressbar_spec = widgets.Label(description="spec")
vcover_progressbar = widgets.IntProgress(min=0, max=1, value=0, description="progress")

calculate_vcover_mode_buttons = widgets.RadioButtons(
    options=['brute force', 'enhanced brute force'],
    value='brute force',
    description='Mode'
)
logger_area = widgets.Textarea(description="log")

box = widgets.VBox([n_input, p_input, widgets.HBox([update_button, connect_button])])
box

In [39]:
class GraphFactory:
    def generate_networkx_matrix(self, n: int, p: float):
        G = nx.Graph()

        for i in range(n):
            G.add_node(i)

        nodes = G.nodes()
        nodes_len = len(nodes)
        for i in range(nodes_len):
            edges = [(i, j) for j in range(i+1, nodes_len) if random.random() <= p]
            G.add_edges_from(edges)

        return G

    def find_subgraphs(self, graph: nx.Graph):
        subgraphs = []
        processed = []

        to_check = [i for i in graph.nodes()]
        stack = []
        
        while True:
            cur_check = to_check[0] if len(to_check) > 0 else None
            if len(stack) == 0:
                if cur_check is None:
                    break

                stack += [cur_check]
                processed += [cur_check]
                to_check.remove(cur_check)

                subgraphs += [[]]
            else:
                cur_check = stack.pop()
                if cur_check not in subgraphs[-1]: # prevent cycles from occuring twice
                    subgraphs[-1] += [cur_check]
                    processed += [cur_check]

                    adding = [n for n in graph.neighbors(cur_check) if n not in processed]
                    stack += adding
                    to_check = [n for n in to_check if n not in adding]
            
        return subgraphs

    def connect(self, graph: nx.Graph):
        subgraphs = self.find_subgraphs(graph)

        first = None
        for sg in subgraphs:
            if first is None:
                first = sg[0]
            else:
                graph.add_edge(first, sg[0])

        return graph

factory = GraphFactory()

In [40]:
out_graph = widgets.Output()
class State:
    displayed_graph = None
    vcover_interrupt = 0

def get_color_map(nodes: list[int], edges: list[(int, int)], k=k_vcover_input.value) -> list[str]:
    isolated = find_isolated_nodes(nodes, edges)
    pendant = find_pendant_nodes(nodes, edges)
    tops = find_tops_nodes(nodes, edges, k)

    color_map = []
    for node in nodes:
        if node in isolated:
            color_map += ['#cccccc']
        elif node in pendant:
            color_map += ['#00bb00']
        elif node in tops:
            color_map += ['#cc0000']
        else:
            color_map += ['#5555dd']
    return color_map

def draw_nx(graph, color_map: list[str] = None):
    if color_map == None:
        color_map = get_color_map(list(graph.nodes()), list(graph.edges()), k_vcover_input.value)

    nx.draw(graph, node_color=color_map, pos=nx.spring_layout(graph, seed = 1))
    nx.draw_networkx_labels(graph, pos=nx.spring_layout(graph, seed = 1))
    k_vcover_input.max = len(graph.nodes())

def on_connect():
    with out_graph:
        dg = State.displayed_graph
        if dg is not None:
            clear_output()
            dg = factory.connect(dg)
            State.displayed_graph = dg
            draw_nx(dg)
            plt.show()

def on_update():
    with out_graph:
        clear_output()
        connect_button.disabled = False
        n = n_input.value
        p = p_input.value
        State.displayed_graph = factory.generate_networkx_matrix(n, p)
        draw_nx(State.displayed_graph)
        plt.show()
          
widgets.VBox([out_graph])

In [41]:
matrix_widget = widgets.Output()

def update_matrix():
    with matrix_widget:
        clear_output()
        result = print_matrix(State.displayed_graph)
        print(result)
        matrix_widget.value = result

def print_matrix(graph: nx.Graph):
    nodes = [n for n in graph.nodes()]
    matrix = []

    for node in nodes:
        adjacent = 0
        neighbors = [n for n in graph.neighbors(node)]

        row = []
        for adjacentNode in nodes:
            score = 0
            if adjacentNode in neighbors:
                if adjacentNode == node:
                    score += 2
                else:
                    score += 1
            row += [score] 
        
        matrix += [row]
    

    matrix_str = '\n'.join(['{:3}'.format(nodes[i]) + '| ' + ''.join(['{:3}'.format(item) for item in matrix[i]]) for i in range(len(matrix))])
    header_str = '     ' + ''.join(['{:3}'.format(item) for item in nodes]) + "\n      " + ''.join(['---' for _ in nodes])

    return 'ADJACENCY MATRIX\n\n' + header_str + "\n" + matrix_str

def on_update_button(_):
    on_update()
    update_matrix()

def on_connect_button(_):
    on_connect()
    update_matrix()

update_button.on_click(on_update_button)
connect_button.on_click(on_connect_button)

widgets.VBox([matrix_widget])

In [42]:
def add_pendant(g):
    nodes = list(g.nodes())
    random.shuffle(nodes)
    edges = list(g.edges())

    for node in nodes:
        adjacent_nodes = find_adjacent_nodes(node, edges)
        deg = len(adjacent_nodes)
        if deg == 0:
            g.add_edge(node, random_node(nodes))
        elif deg > 1:
            to_remove = deg - 1
            for i in range(to_remove):
                g.remove_edge(node, adjacent_nodes[i])
        else: continue
        break

def remove_pendant(g):
    nodes = list(g.nodes())
    random.shuffle(nodes)
    edges = list(g.edges())

    for node in nodes:
        deg = degree(node, edges)
        if deg == 1:
            g.add_edge(node, random_node(nodes, not_in=[node]))
            break

def add_tops(g):
    nodes = list(g.nodes())
    random.shuffle(nodes)
    edges = list(g.edges())

    k = k_vcover_input.value

    for node in nodes:
        adjacent_nodes = find_adjacent_nodes(node, edges)
        deg = len(adjacent_nodes)
        if deg < k:
            to_add = (k+1) - deg

            nodes_pickfrom = [n for n in nodes if n not in adjacent_nodes and n != node]
            random.shuffle(nodes_pickfrom)
            for i in range(to_add):
                g.add_edge(node, nodes_pickfrom[i])
            break

def remove_tops(g):
    nodes = list(g.nodes())
    random.shuffle(nodes)
    edges = list(g.edges())

    k = k_vcover_input.value

    for node in nodes:
        adjacent_nodes = find_adjacent_nodes(node, edges)
        deg = len(adjacent_nodes)
        if deg > k:
            removing = int(random.random() * deg)
            for i in range(removing):
                g.remove_edge(node, adjacent_nodes[i])
            break

def refresh_outputs(color_map = []):
    with out_graph:
        dg = State.displayed_graph
        if dg is not None:
            clear_output()
            draw_nx(dg, color_map=color_map)
            plt.show()
    update_matrix()

def refresh_with_coloring():
    g = State.displayed_graph
    color_map = get_color_map(list(g.nodes()), list(g.edges()), k_vcover_input.value)
    refresh_outputs(color_map=color_map)

def on_pendant_plus_button(_):
    add_pendant(State.displayed_graph)
    refresh_with_coloring()

def on_pendant_minus_button(_):
    remove_pendant(State.displayed_graph)
    refresh_with_coloring()

def on_tops_plus_button(_):
    add_tops(State.displayed_graph)
    refresh_with_coloring()

def on_tops_minus_button(_):
    remove_tops(State.displayed_graph)
    refresh_with_coloring()

def on_highlight_kernelization_button(_):
    refresh_with_coloring()

pendant_plus_button.on_click(on_pendant_plus_button)
pendant_minus_button.on_click(on_pendant_minus_button)
tops_plus_button.on_click(on_tops_plus_button)
tops_minus_button.on_click(on_tops_minus_button)
highlight_kernelization_button.on_click(on_highlight_kernelization_button)


In [43]:
def on_update_button(_):
    on_update()
    update_matrix()
    calculate_k_vcover_button.disabled = False

update_button.on_click(on_update_button)

def on_update_k_slider(_):
    refresh_with_coloring()

def update_progressbar(x):
    val = vcover_progressbar.value
    max = vcover_progressbar.max
    quo = val/max*100

    vcover_progressbar.value = x+val
    progressbar_spec.value = "%.2f" % quo + "% (" + str(val) + "/" + str(max) + ")"

def on_calculate_k_vcover_button(_):
    calculate_k_vcover_button.disabled = True
    if State.displayed_graph == None:
        return

    g = State.displayed_graph
    nodes = list(g.nodes())
    edges = list(g.edges())
    k = k_vcover_input.value
    n = len(nodes)
    enhanced_mode = calculate_vcover_mode_buttons.value == "enhanced brute force"
    cover = [False for _ in range(n)]

    vcover_progressbar.value = 0
    vcovers = None

    if enhanced_mode:
        calculated_text.value = "Kernalizing vertices..."
        (ns, es, nodes_in_cover) = kernalize_vertices(nodes, edges, k=k)

        if ns is not None:
            calculated_text.value = "Kernalized vertices"
            logger_area.value = str(ns) + ";" + str(es)
            n = len(ns)
            cover = [False for _ in range(n)]
            
            i = 0
            l = lambda p : update_progressbar(p)
            vcover_progressbar.max = 2**(k+2)

            calculated_text.value = "Calculating (enhanced)..."
            vcovers = find_vcovers(ns, nodes, edges, cover, n, i, k, l)

            calculated_text.value = "Assembling..."
            for i in range(len(vcovers)):
                vcover = vcovers[i]
                vcover += nodes_in_cover
                vcovers[i] = sorted(vcover)

    else:
        i = 0
        l = lambda p : update_progressbar(p)
        vcover_progressbar.max = 2**(k+2)

        calculated_text.value = "Calculating..."
        vcovers = find_vcovers(nodes, nodes, edges, cover, n, i, k, l)
    

    if vcovers == None or len(vcovers) == 0:
        calculated_text.value = "No v-cover possible for k=" + str(k)
    else:
        calculated_text.value = "Possible v-covers: \n" + str(vcovers)
        vcover = vcovers[0]
        calculated_one_text.value = str(vcover)
    
    calculate_k_vcover_button.disabled = False

calculate_k_vcover_button.on_click(on_calculate_k_vcover_button)

box_week2 = widgets.VBox([
    widgets.HBox([k_vcover_input, highlight_kernelization_button]), 
    widgets.HBox([calculate_vcover_mode_buttons, calculate_k_vcover_button]), 
    widgets.HBox([calculated_text, calculated_one_text]), 
    vcover_progressbar, 
    progressbar_spec,
    widgets.HBox([pendant_plus_button, pendant_minus_button]),
    widgets.HBox([tops_plus_button, tops_minus_button])
])
cpanel = widgets.HBox([
    box_week2,
    widgets.VBox([
        logger_area
    ])
])
cpanel